In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver ## 브라우저 제어
from selenium.webdriver.chrome.service import Service as ChromeService ## 크롬 브라우저 제어
from selenium.webdriver.common.keys import Keys ## 키보드 제어
from selenium.webdriver.support import expected_conditions as EC ## 웹페이지 조건 기다림
from selenium.webdriver.support.ui import WebDriverWait ## 웹페이지 대기
from selenium.webdriver.common.by import By ## 웹페이지 요소

from webdriver_manager.chrome import ChromeDriverManager


import warnings
warnings.filterwarnings(action="ignore")
import re
import pandas as pd
import datetime
import time
import os
from tqdm import tqdm

today = datetime.date.today().strftime("%Y-%m-%d")

options = webdriver.ChromeOptions()
options.add_argument("--disable-web-security") ## 웹 보안 해제
options.add_argument("--disable-site-isolation-trials") ## 사이트 격리 해제
options.add_argument("--window-size=1440,1200") ## 창 크기

# options.add_argument('headless') ## 헤드리스

Keys = Keys()
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2564.116 Safari/537.37"
) ## 가짜 User-Agent 실행 (크롬 49 버전으로 위장)


driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install()), options=options
) ## 크롬드라이버를 설치하고 옵션을 적용하여 실행

driver.implicitly_wait(10) ## 웹페이지가 로드되는 시간을 암묵적 대기, 5초 기다림



In [2]:
BASE_URL = '''https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D&tab=position'''

driver.get(BASE_URL)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [3]:
time.sleep(1)

## 무한 스크롤 아래로 내리기
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height



### 웹 페이지는 업데이트 될때마다 구조가 변경되므로 코드가 작동하지 않을 수 있어요.

- 그림에 보이는 요소를 찾아주세요.
![원티드](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%90%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_1.png?alt=media&token=19d79b2f-96f1-445d-8905-961f71b41854)

In [8]:
## 페이지 소스 가져오기
htmls = driver.page_source
soups = BeautifulSoup(htmls, 'html.parser')

items= soups.find_all('div', class_='JobCard_container__REty8 JobCard_container--variant-card__gaJS_')


In [9]:
len(items)

137

In [10]:
## index를 유지하면서 중복되는 요소 제거
items = list({item.text: item for item in items}.values())


In [11]:
len(items)

128

In [12]:
target_url_list = []
for item in items:
    url = 'https://www.wanted.co.kr'+item.find('a')['href']
    target_url_list.append(url)

In [13]:
len(target_url_list)

128

## 세부 화면 데이터 수집
- 원하는 회사 공고를 클릭해주세요.
#### ✅ 이런 회사 사이트로 오셔야합니다.

![상세페이지](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%90%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_2.png?alt=media&token=8235bc8b-6329-47c7-b111-4bceee7b5997)

- 마찬가지로 수집을 원하는 곳의 XPATH를 가져와야합니다.

![클래스찾기](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%90%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_3.png?alt=media&token=ca335967-df8f-4b3b-b853-30e468bcb16f)

In [16]:
company_name = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
title = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/h1').text

In [19]:
company_name, title

('윌로그', '데이터 분석 담당자 (3년 이상)')

### 더보기 버튼 클릭

![더보기](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%90%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_4.png?alt=media&token=9a99ed4f-5e3a-44d8-8450-b17f26620dfc)

# 🚨 주의, 더보기 버튼이 클릭된 상태에서 아래의 코드를 실행하면 당연히 에러가 터집니다.

In [25]:
## 상세정보 클릭
driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()


#### 계속 말씀드리지만, 각각 태그에 맞는 Class명을 찾아서 넣어주셔야합니다.
- 홈페이지가 업데이트 되면 Class명이 변경될 수 있어요.

![태그에맞는클래스](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%90%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_5.png?alt=media&token=453e102e-ece8-4ea7-a687-94205ff37e1f)

In [26]:
## 페이지 소스 업데이트
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__2JNh_'})\
                    .find('article', {'class': 'JobDescription_JobDescription__dq8G5'})



#### 포지션 상세

- 팀 소개
- 주요업무
- 자격요건
- 우대사항
- 혜택 및 복지

In [28]:
position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__2JNh_'})\
                    .find('article', {'class': 'JobDescription_JobDescription__dq8G5'})

team_info = position_detail.find('p').text
main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[0].find('h3').text
main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[0].find('p').text

In [30]:
requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('h3').text
requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('p').text


In [33]:
requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('h3').text
requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('p').text

preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[2].find('h3').text
preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[2].find('p').text
wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[3].find('h3').text
wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[3].find('p').text

### 기술 스택

In [34]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# JobSkillTags_JobSkillTags__V1tZr
skill_stacks = soup.find('article', {'class': 'JobSkillTags_JobSkillTags__UA0s6'})

skill_title = skill_stacks.find('h2').text
skill_list = [i.text for i in skill_stacks.find('ul').find_all('li')]


#### 직장 주소

In [39]:
address = soup.find('article', {'class': 'JobWorkPlace_JobWorkPlace__Q6Gml'}).find('span').text
address

'서울특별시 강남구 삼성로 507, JS타워 9층'

In [40]:
def detail_crawl(driver, idx):

    team_info = None
    main_job_title = None
    main_job_content = None
    requirement_title = None
    requirement_content = None
    preferred_title = None
    preferred_content = None
    wellbeing_title = None
    wellbeing_content = None
    skill_title = None
    skill_list = None
    address = None
    current_url = None

    ## 상세 더보기 버튼 누르기
    try:
        ## 상세정보 클릭
        driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()

    except Exception as e:
        print(idx, e)
        pass

    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    company_name = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
    title = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/h1').text
    try:
        target_elements1 = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')
        # 요소가 화면에 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView();", target_elements1)

        # 요소가 보일때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,\
        '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')))

        ## 스크린샷 찍기
        driver.save_screenshot(f'screenshot/{idx}_{company_name}.png')

        ## 저장할때까지 기다리기
        time.sleep(1)
    except Exception as e:
        print("스크린샷 에러:", idx, e)

    try:
        position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__2JNh_'})\
                            .find('article', {'class': 'JobDescription_JobDescription__dq8G5'})

        team_info = position_detail.find('p').text
        main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[0].find('h3').text
        main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[0].find('p').text
        requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('h3').text
        requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[1].find('p').text

        preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[2].find('h3').text
        preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[2].find('p').text
        wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[3].find('h3').text
        wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Lhegj'})[3].find('p').text
        ## 기술스택
        skill_stacks = soup.find('article', {'class': 'JobSkillTags_JobSkillTags__UA0s6'})
        if skill_stacks:
            skill_title = skill_stacks.find('h2').text
            skill_list = [i.text for i in skill_stacks.find('ul').find_all('li')]

        address = soup.find('article', {'class': 'JobWorkPlace_JobWorkPlace__Q6Gml'}).find('span').text
        current_url = driver.current_url
    except Exception as e:
        print("상세정보 에러:", idx, e)
        pass

    return company_name, title, team_info, main_job_title, main_job_content, requirement_title, requirement_content, preferred_title, preferred_content, wellbeing_title, wellbeing_content, skill_title, skill_list, address, current_url


In [41]:
target_url_list

['https://www.wanted.co.kr/wd/225741',
 'https://www.wanted.co.kr/wd/225236',
 'https://www.wanted.co.kr/wd/225354',
 'https://www.wanted.co.kr/wd/224969',
 'https://www.wanted.co.kr/wd/161548',
 'https://www.wanted.co.kr/wd/225142',
 'https://www.wanted.co.kr/wd/224170',
 'https://www.wanted.co.kr/wd/225115',
 'https://www.wanted.co.kr/wd/224337',
 'https://www.wanted.co.kr/wd/223481',
 'https://www.wanted.co.kr/wd/225045',
 'https://www.wanted.co.kr/wd/201396',
 'https://www.wanted.co.kr/wd/218916',
 'https://www.wanted.co.kr/wd/208247',
 'https://www.wanted.co.kr/wd/85810',
 'https://www.wanted.co.kr/wd/222878',
 'https://www.wanted.co.kr/wd/220513',
 'https://www.wanted.co.kr/wd/225195',
 'https://www.wanted.co.kr/wd/223175',
 'https://www.wanted.co.kr/wd/215835',
 'https://www.wanted.co.kr/wd/203738',
 'https://www.wanted.co.kr/wd/212760',
 'https://www.wanted.co.kr/wd/216979',
 'https://www.wanted.co.kr/wd/219963',
 'https://www.wanted.co.kr/wd/142793',
 'https://www.wanted.co.kr

In [43]:
print("✅ 크롤링 시작")
data_list = []
for idx, target in enumerate(tqdm(target_url_list)):
    try :
        driver.get(target)
        company_name, title, team_info, main_job_title, main_job_content, requirement_title, requirement_content, preferred_title, preferred_content, wellbeing_title, wellbeing_content, skill_title, skill_list, address, current_url = detail_crawl(driver, idx)
        time.sleep(1)
        data = {
            'company_name': company_name,
            'title': title,
            'team_info': team_info,
            'Main Job Title': main_job_title,
            'Main Job Content': main_job_content,
            'Requirement Title': requirement_title,
            'Requirement Content': requirement_content,
            'Preferred Title': preferred_title,
            'Preferred Content': preferred_content,
            'Wellbeing Title': wellbeing_title,
            'Wellbeing Content': wellbeing_content,
            'Skill Title': skill_title,
            'Skill List': skill_list,
            'Address': address,
            'Current URL': current_url
        }
        data_list.append(data)
    except Exception as e:
        print(company_name, e)
        pass    

print("✅ 수집완료")
res_df = pd.DataFrame(data_list)


✅ 크롤링 시작


 80%|███████▉  | 102/128 [07:52<02:03,  4.75s/it]

상세정보 에러: 102 list index out of range


100%|██████████| 128/128 [09:54<00:00,  4.65s/it]

✅ 수집완료


In [44]:
### data_list의 '-'를 *로 변경
res_df = res_df.replace('- ', '• ', regex=True)
res_df

,company_name,title,team_info,Main Job Title,Main Job Content,Requirement Title,Requirement Content,Preferred Title,Preferred Content,Wellbeing Title,Wellbeing Content,Skill Title,Skill List,Address,Current URL
0,자비스앤빌런즈(삼쩜삼),Data Analyst,"[We make your Money, save your Time, find you...",주요업무,• 삼쩜삼이 서비스하는 프로덕트 전반에 대한 오너십을 갖고 분석업무를 진행하게 됩니...,자격요건,"• 합리적, 논리적 사고능력을 가지신 분• A/B 테스트에 대한 이해를 바탕으로 실...",우대사항,• 머신러닝에 대한 이해도가 있으신 분[서류는 이렇게 제출해 주세요.]• 지원서류 ...,혜택 및 복지,[최고의 인재에게 최고의 보상]• 개인 성과 및 경영 성과에 따른 인센티브 제도• ...,None,None,"서울 강남구 테헤란로44길 8, 아이콘역삼 9,10층 자비스앤빌런즈",https://www.wanted.co.kr/wd/225741
1,테크랩스,마케팅 데이터분석 (브랜드엑셀러레이팅사업부/데이터분석팀),"""One for all, All for one!""애드테크의 패러다임을 함께 선도할 ...",주요업무,[브랜드 엑셀러레이팅 사업부]ㆍ고관여 제품이나 서비스의 마케팅 전략과 실행능력을 특...,자격요건,• 중상급 수준의 Excel 활용이 가능하신 분(필수)• 종합적 사고에 기반한 방법...,우대사항,• 사회과학 계열 통계를 학습하신 분• 데이터 분석 관련 자격증(사회조사분석사 등)...,혜택 및 복지,"• 자유로운 사내문화: 상호존중, 배려, 자유로운 출퇴근 복장• 자유로운 연차: 연...",None,None,"강남구 언주로 428, 4층, 5층, 6층, 9층, 13층, 14층",https://www.wanted.co.kr/wd/225236
2,코인원(coinone),Product Data Analyst (Business Analyst),Bringing Blockchain into the world.코인원은 블록체인을 ...,주요업무,"• 거래소 앱/웹의 사용자 데이터 분석, 주간 행동 지표 및 인사이트 발표, 중-장...",자격요건,"• 모바일/웹 플랫폼, 유사 업종의 데이터 분석가 실무 3년 이상 경력이 있는 분 ...",우대사항,"• 직책 없이도 주위의 인정을 받을 만큼 강한 오너십과 책임감, 봉사 의식이 강한 ...",혜택 및 복지,"혜택 및 복지[코인원 Crew만을 위한 복리후생]# 든든한 밥심을 위해•﻿ 점식,...",None,None,"서울특별시 영등포구 여의대로 108 파크원 타워1 45층, 46층",https://www.wanted.co.kr/wd/225354
3,오피앤,[Product 팀] Data Analyst,오픈닥터는 병의원 맞춤형 부동산 중개 및 의료 상권 정보를 제공하는 프롭테크 스타트...,주요업무,[팀 소개]• 오픈닥터의 Data Analyst는 Product 팀 소속으로 일합니...,자격요건,"• SQL, R, Python 등의 DBMS와 통계 소프트웨어를 잘 다루시는 분• ...",우대사항,• Data 관련 전공 석사 이상• Data 관련 프로젝트나 조직을 리드하신 경험이...,혜택 및 복지,문제 해결을 잘하는 훌륭한 팀이 실질적 성과에 집중하고 있어요! 일 잘하는 팀원들만...,None,None,"서울시 강남구 논현로150길 40, 3층",https://www.wanted.co.kr/wd/224969
4,포티투닷(42dot),Senior Data Analyst,[포지션 소개]42dot의 Data Analyst는 SDV (Software Def...,주요업무,• Data를 통한 비즈니스 의사 결정 지원• 주요 비즈니스 지표를 설계/개발• 비...,자격요건,• 데이터 분석 프로젝트 수행 및 구축 경험 7년 이상• 분석 프로젝트에 대한 요구...,우대사항,"• 컴퓨터 공학, 통계학, 산업공학 등 관련 학위 보유• 차량 데이터에 대한 이해 ...",혜택 및 복지,[전형 절차]∙ 서류전형 • 1차 인터뷰 • 2차 인터뷰 • 3차 인터뷰 • 처우협...,기술 스택 • 툴,"[Python, SQL, Tableau]","서울특별시 강남구 남부순환로 2621 (도곡동, 서브원 강남빌딩) 9층",https://www.wanted.co.kr/wd/161548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,당근마켓,데이터 분석가 • 검색,"Data Analyst를 소개해요당근마켓의 비전은 ""당근마켓은 동네 이웃 간의 연결...",주요업무,"• 제품 개선과 사용자 경험 개선에 필요한 데이터 분석 과제를 정의하고, 이를 바탕...",자격요건,• 5년 이상의 모바일 데이터 분석 경험이 있으며 데이터에 근거한 인사이트를 기반으...,우대사항,"• 검색, 피드 등 유저의 발견 경험을 연관된 데이터를 활용해 개선해본 경험이 있으...",혜택 및 복지,"업무 제도 및 지원 • 세미나, 스터디 참석을 권장하고 비용을 지원하고 있어요...",None,None,"서울시 서초구 강남대로 465, 교보타워 10,11,12층",https://www.wanted.co.kr/wd/179347
124,현대캐피탈,Data Scientist,"현대캐피탈은 자동차금융을 중심으로 신용대출, 주택대출 등 주요 금융상품과 서비스를 ...",주요업무,• 현대캐피탈 앱/웹의 자동차 및 금융서비스를 위한 AI/ML기반 개인화 모델 개발...,자격요건,"• Tensorflow, PyTorch 등을 이용한 AI/ML모델 개발 경험• On...",우대사항,"• AI Engineering, ML-Ops에 대한 이해와 경험• AI/ML관련 학...",혜택 및 복지,• 주택구입/전세자금대출지원• 차량구입지원금 : 현대기아차 구매시 Payback형식...,기술 스택 • 툴,"[Oracle, Java, SQL, Spring Framework]",현대캐피탈 본사(서울 중구 세종대로 14 그랜드센트럴 A동),https://www.wanted.co.kr/wd/193867
125,무신사,Data Analyst (데이터프로덕트팀),[MUSINSA] Data Analyst무신사는 2001년 온라인 커뮤니티로 시작해...,주요업무,• 추천/검색 프로덕트 분석을 바탕으로 추천/검색 모델 방향성 제시• 유저에 대한 ...,자격요건,"• DA경력 최소 4년 이상이신 분• SQL, 파이썬 등을 활용하여 데이터 가공/추...",우대사항,• 커머스 도메인에서 서비스 분석을 진행해 본 분• 데이터를 집요하게 파고들 수 있...,혜택 및 복지,[근무환경]• 8시~11시 자율 출퇴근• 월 1회 Early Friday 운영 (매...,기술 스택 • 툴,"[Android, iOS, MySQL, React, JavaScript, Kotli...",서울특별시 성동구 성수동2가 277-47 무신사캠퍼스N1,https://www.wanted.co.kr/wd/181976
126,교보문고,데이터 분석 컨설턴트,[데이터인텔리전스팀]교보문고는 대한민국의 지식문화를 선도하는 기업으로 전국 37개 ...,주요업무,"데이터분석과 커뮤니케이션에 대한 전문성을 바탕으로,1. 데이터 기반 문제해결 ...",자격요건,• SQL 기반의 데이터 분석 및 쿼리문 작성에 익숙하신 분• Tableau 데이터...,우대사항,• 데이터 분석 솔루션이나 플랫폼 구축 프로젝트 참여 경험이 있는 분• 온오프라인 ...,혜택 및 복지,• 원하는 시간에 출근할 수 있는 '시차출퇴근제'• 사내 사이버연수원 운영 및 직무...,None,None,"서울특별시 마포구 월드컵북로54길 25 (상암동), DMC푸르지오시티 7층 교보문고...",https://www.wanted.co.kr/wd/149962


In [16]:
driver.quit() ## 크롬드라이버 종료

In [24]:

team_info = None
main_job_title = None
main_job_content = None
requirement_title = None
requirement_content = None
preferred_title = None
preferred_content = None
wellbeing_title = None
wellbeing_content = None
skill_title = None
skill_list = None
address = None
current_url = None


## 상세 더보기 버튼 누르기
try:
    driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()
except Exception as e:
    print(idx, e)
    pass

time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [25]:

company_name = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
title = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/h1').text
try:
    target_elements1 = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')
    # 요소가 화면에 보이도록 스크롤
    driver.execute_script("arguments[0].scrollIntoView();", target_elements1)

    # 요소가 보일때까지 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,\
    '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')))

    ## 스크린샷 찍기
    driver.save_screenshot(f'screenshot/test.png')

    ## 저장할때까지 기다리기
    time.sleep(1)
except Exception as e:
    print("스크린샷 에러:", idx, e)




In [26]:
position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                    .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})
team_info = position_detail.find('p').text
main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('h3').text
main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('p').text
requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('h3').text
requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('p').text

preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('h3').text
preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('p').text
wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('h3').text
wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('p').text

## 기술스택
skill_stacks = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                    .find('article', {'class': 'JobSkillTags_JobSkillTags__V1tZr'})

if skill_stacks:

    skill_title = skill_stacks.find('h2').text
    skill_list = [i.text for i in skill_stacks.find('ul').find_all('li')]

address = soup.find('article', {'class': 'JobWorkPlace_JobWorkPlace__5fs5x'}).find('span').text
current_url = driver.current_url


IndexError: list index out of range

In [27]:
preferred_title

'혜택 및 복지'

In [28]:
preferred_content

'[Recruiting Process]Application Review - Home Test - Phone Interview - Virtual Onsite Interview – Offer'

In [18]:
import secrets

secrets.token_hex()

'98be3e798d86de55001c047c51e6aab48fc785df6d7197d219f55db7f58f196f'

In [19]:
secrets.token_hex()

'a6fb045e9c73c018cb0ba456eaf2edf77b033cda0e983795de0e26dca9782784'